In [10]:
import os
import pandas as pd

print("Current working directory:", os.getcwd())  # Check your working directory

# Ensure the file exists before reading
file_path = "/home/deepak-bhagat/Downloads/data_odometry_poses/dataset/poses/00.txt"

if os.path.exists(file_path):
    poses = pd.read_csv(file_path, delim_whitespace=True, header=None)  # Use whitespace delimiter for space-separated files
    print(poses.describe())
else:
    print(f"Error: File not found at {file_path}")


Current working directory: /home/deepak-bhagat/software/Autonomous/Slamandpath
                0            1            2            3            4   \
count  4541.000000  4541.000000  4541.000000  4541.000000  4541.000000   
mean      0.077408     0.034070    -0.012741    22.378433     0.009753   
std       0.691782     0.024297     0.716764   142.014459     0.041641   
min      -0.999939    -0.066186    -0.999054  -271.280600    -0.081747   
25%      -0.584706     0.016139    -0.875506   -59.754040    -0.023489   
50%      -0.012153     0.033974    -0.002484    10.714580     0.005515   
75%       0.895636     0.053932     0.741525   132.663600     0.038189   
max       1.000000     0.093741     0.999995   292.239500     0.110780   

                5            6            7            8            9   \
count  4541.000000  4541.000000  4541.000000  4541.000000  4541.000000   
mean      0.998450     0.014748    -7.904195     0.013595     0.027745   
std       0.001302     0.032392 

/tmp/ipykernel_1365039/2491994652.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  poses = pd.read_csv(file_path, delim_whitespace=True, header=None)  # Use whitespace delimiter for space-separated files


In [11]:
poses.head


<bound method NDFrame.head of             0             1             2             3             4   \
0     1.000000  9.043680e-12  2.326809e-11  5.551115e-17  9.043683e-12   
1     0.999998  5.272628e-04 -2.066935e-03 -4.690294e-02 -5.296506e-04   
2     0.999991  1.048972e-03 -4.131348e-03 -9.374345e-02 -1.058514e-03   
3     0.999980  1.566466e-03 -6.198571e-03 -1.406429e-01 -1.587952e-03   
4     0.999964  2.078471e-03 -8.263498e-03 -1.874858e-01 -2.116664e-03   
...        ...           ...           ...           ...           ...   
4536  0.998794  3.096949e-03 -4.901132e-02 -5.369648e+00 -3.267539e-03   
4537  0.998841 -1.632819e-03 -4.811549e-02 -5.418986e+00  1.437775e-03   
4538  0.998874 -4.879842e-03 -4.718123e-02 -5.470861e+00  4.550713e-03   
4539  0.998887 -7.351853e-03 -4.658741e-02 -5.524006e+00  6.830398e-03   
4540  0.998909 -9.331753e-03 -4.575093e-02 -5.583931e+00  8.633629e-03   

            5             6             7             8             9   \
0     1

In [14]:
import pykitti
import numpy as np
import matplotlib.pyplot as plt

# Set the base directory where your KITTI odometry dataset is stored
basedir ="/home/deepak-bhagat/Downloads/data_odometry_calib/dataset/"

# Specify the sequence you have (e.g., '00')
sequence = '00'

# If you only have a subset of frames, you can specify it. Otherwise, set frames=None.
frames = None

# Create an instance of the odometry class
data = pykitti.odometry(basedir, sequence)

# Since you don't have image files, avoid calling image-related methods.
# Instead, work with LiDAR, calibration, timestamps, and poses.

# 1. Print basic information
print("Number of frames:", len(data))
print("Calibration data for camera 0 (intrinsics):\n", data.calib.K_cam0)
print("First timestamp:", data.timestamps[0])
if data.poses:
    print("First ground truth pose:\n", data.poses[0])
else:
    print("No ground truth poses available.")

# 2. Access and visualize the first LiDAR scan
velo_scan = data.get_velo(0)
print("Shape of first LiDAR scan:", velo_scan.shape)  # Expected shape: (N, 4)

# Plot a top-down view of the LiDAR scan (using x and y coordinates, color coded by reflectance)
plt.figure(figsize=(8, 6))
plt.scatter(velo_scan[:, 0], velo_scan[:, 1], s=0.5, c=velo_scan[:, 3], cmap='jet')
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.title("Top-Down View of the First LiDAR Scan")
plt.colorbar(label="Reflectance")
plt.show()

# 3. If you want to iterate over a few LiDAR scans and check their timestamps or poses:
for idx, (velo, timestamp) in enumerate(zip(data.velo, data.timestamps)):
    print(f"Frame {idx}: timestamp = {timestamp}")
    # Optionally, do further processing with the LiDAR data
    if idx >= 2:  # Process only the first three scans for this example
        break

# 4. Using calibration to transform a point from the velodyne frame to camera 0 frame:
# Create a homogeneous point (in the velodyne coordinate system)
point_velo = np.array([0, 0, 0, 1])
point_cam0 = data.calib.T_cam0_velo.dot(point_velo)
print("Point in camera 0 coordinate system:", point_cam0)


Ground truth poses are not avaialble for sequence 00.
Number of frames: 4541
Calibration data for camera 0 (intrinsics):
 [[718.856    0.     607.1928]
 [  0.     718.856  185.2157]
 [  0.       0.       1.    ]]
First timestamp: 0:00:00
No ground truth poses available.


IndexError: list index out of range

In [ ]:
import pykitti
import numpy as np
import open3d as o3d

# Set the base directory and sequence for the odometry dataset
basedir = 'D:\DSGRMS\KIITI DataSet\dataset'
sequence = '00'
frames = None  # Load all frames or specify a range, e.g., range(0, 50)

# Create an instance of the odometry dataset
data = pykitti.odometry(basedir, sequence, frames=frames)

# Load the first LiDAR scan from the dataset
velo_scan = data.get_velo(6)  # shape: (N, 4) -> [x, y, z, reflectance]

# Extract the 3D points (x, y, z)
points = velo_scan[:, :3]

# Use reflectance for color (scaled to [0,1])
reflectance = velo_scan[:, 3]
colors = np.tile(reflectance.reshape(-1, 1), (1, 3))
colors = colors / np.max(colors)  # Normalize to range [0,1]

# Create an Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


In [ ]:
velo_scan = data.velo

In [ ]:
next(velo_scan)

In [ ]:
data.timestamps[4540]

In [ ]:
import numpy as np
import open3d as o3d
import pykitti
import copy

# --- SLAM Setup ---
basedir ="D:\DSGRMS\KIITI DataSet\dataset"  # Replace with your path
sequence = '00'
data = pykitti.odometry(basedir, sequence, frames=range(0, 10))  # For demo, use first 10 scans

def preprocess_point_cloud(pcd, voxel_size=0.1):
    """ Downsample and estimate normals """
    pcd_down = pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30)
    )
    return pcd_down

# Initialize with the first LiDAR scan
velo_scan = data.get_velo(0)
pcd_prev = o3d.geometry.PointCloud()
pcd_prev.points = o3d.utility.Vector3dVector(velo_scan[:, :3])
pcd_prev = preprocess_point_cloud(pcd_prev, voxel_size=0.1)

transformation_global = np.eye(4)
trajectory = [transformation_global.copy()]
global_map = copy.deepcopy(pcd_prev)

# SLAM Loop (using ICP to register consecutive scans)
for i in range(1, 10):
    velo_scan = data.get_velo(i)
    pcd_curr = o3d.geometry.PointCloud()
    pcd_curr.points = o3d.utility.Vector3dVector(velo_scan[:, :3])
    pcd_curr = preprocess_point_cloud(pcd_curr, voxel_size=0.1)
    
    # ICP Registration (point-to-point)
    threshold = 1.0  # max correspondence distance
    icp_result = o3d.pipelines.registration.registration_icp(
        pcd_curr, pcd_prev, threshold, np.eye(4),
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    
    # Update global transformation and trajectory
    transformation_global = transformation_global @ icp_result.transformation
    trajectory.append(transformation_global.copy())
    
    # Transform current point cloud into the global coordinate frame
    pcd_curr_transformed = copy.deepcopy(pcd_curr)
    pcd_curr_transformed.transform(transformation_global)
    global_map += pcd_curr_transformed
    global_map = global_map.voxel_down_sample(0.1)
    
    # Prepare for next iteration
    pcd_prev = pcd_curr
    print(f"Processed frame {i}")

# Visualize the global map and trajectory
print("Estimated Trajectory Poses:")
for pose in trajectory:
    print(pose)

o3d.visualization.draw_geometries([global_map])
